In [23]:
import requests
import datetime
import pandas as pd
import os

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric, DatasetMissingValuesMetric, ColumnQuantileMetric

from joblib import load, dump
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

ImportError: cannot import name 'NumericalStatistics' from 'evidently.metrics' (c:\Users\HP\anaconda3\envs\MLO\Lib\site-packages\evidently\metrics\__init__.py)

In [13]:
# Definir la ruta del archivo parquet
file_path = r"C:\Users\HP\Documents\Python_Projects\mlops-zoomcamp\05-monitoring\green_tripdata_2024-03.parquet"

# Definir la URL para descargar el archivo
url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-03.parquet"

# Definir el directorio de destino para guardar el archivo
save_dir = './data'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Definir la ruta completa de destino
save_path = os.path.join(save_dir, os.path.basename(file_path))

# Descargar el archivo si no existe localmente
if not os.path.exists(save_path):
    print(f"Downloading {url} to {save_path}...")
    resp = requests.get(url, stream=True)
    with open(save_path, "wb") as handle:
        for data in tqdm(resp.iter_content(chunk_size=1024),
                         desc=f"Downloading {os.path.basename(file_path)}",
                         total=int(resp.headers.get('content-length', 0)) // 1024,
                         unit='KB'):
            handle.write(data)
else:
    print(f"{os.path.basename(file_path)} already exists locally.")

# Leer el archivo parquet descargado
df = pd.read_parquet(save_path)

# Obtener la forma del dataframe
shape = df.shape

print(f"The shape of the downloaded data is: {shape}")
print(f"There are {shape[0]} rows in the data.")

green_tripdata_2024-03.parquet already exists locally.
The shape of the downloaded data is: (57457, 20)
There are 57457 rows in the data.


In [17]:
df.describe()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
count,57457.000000,57457,57457,55360.000000,57457.000000,57457.000000,55360.000000,57457.000000,57457.000000,57457.000000,57457.000000,57457.000000,57457.000000,0.0,57457.000000,57457.000000,55360.000000,55353.000000,55360.000000
mean,1.877334,2024-03-16 04:02:52.405399,2024-03-16 04:21:00.076039,1.179986,95.524688,138.629149,1.309538,13.522828,17.313474,0.904472,0.577410,2.386255,0.192537,NaN,0.979378,22.904832,1.321062,1.038047,0.737730
min,1.000000,2008-12-31 23:02:24,2008-12-31 23:02:30,1.000000,1.000000,1.000000,0.000000,0.000000,-295.080000,-2.500000,-0.500000,-1.560000,0.000000,NaN,-1.000000,-296.080000,1.000000,1.000000,-2.750000
25%,2.000000,2024-03-08 13:53:56,2024-03-08 14:13:49,1.000000,74.000000,74.000000,1.000000,1.100000,9.300000,0.000000,0.500000,0.000000,0.000000,NaN,1.000000,13.440000,1.000000,1.000000,0.000000
50%,2.000000,2024-03-15 22:49:01,2024-03-15 23:09:52,1.000000,75.000000,138.000000,1.000000,1.790000,13.500000,0.000000,0.500000,2.000000,0.000000,NaN,1.000000,18.500000,1.000000,1.000000,0.000000
75%,2.000000,2024-03-23 20:11:25,2024-03-23 20:34:48,1.000000,97.000000,220.000000,1.000000,3.100000,19.800000,1.000000,0.500000,3.610000,0.000000,NaN,1.000000,27.050000,2.000000,1.000000,2.750000
max,2.000000,2024-04-01 00:01:45,2024-04-01 16:11:00,99.000000,265.000000,265.000000,9.000000,125112.200000,841.600000,10.000000,4.250000,150.000000,26.760000,NaN,1.000000,856.980000,5.000000,2.000000,2.750000
std,0.328056,NaN,NaN,1.356719,57.285088,76.295346,0.967749,770.416255,14.958249,1.382446,0.366916,3.159273,1.184551,NaN,0.154253,17.013735,0.497858,0.191311,1.218039


TypeError: ColumnMapping.__init__() got an unexpected keyword argument 'target_col'